# Grouping and Aggregation

One of the most powerful capabilities of Pandas is its ability to perform fast group-wise operations on data. Consider a question like "What is the average fare paid by passengers in each class?" or "How do survival rates differ by gender?" These questions underpins much of exploratory data analysis and requires splitting data into groups, computing within each group, and combining results. Pandas provides flexible and intuitive tools for exactly these kinds of operations, and in this section we will explore them using real-world datasets that illuminate when and why these tools are so valuable.

## The Split-Apply-Combine Pattern

At the core of Pandas grouping operations lies the *split-apply-combine* pattern. This conceptual model, introduced by Hadley Wickham in his foundational work on the R programming language, can be thought of as a three-step process that mirrors how we naturally think about grouped computations. First, the data is *split* into groups based on some criterion—perhaps passenger class or port of embarkation. Then a function is *applied* independently to each group to compute summary statistics or transformations. Finally, the results are *combined* back into a single data structure that presents the findings coherently.

You can think of this a bit like organizing a stack of papers by department, having each department calculate their own totals, and then collecting all the summary sheets back into one report. The beauty of this pattern is that it allows us to express complex analyses concisely—Pandas automates the entire workflow through its `groupby` functionality, handling the bookkeeping of group membership and result assembly behind the scenes.

The following ASCII diagram illustrates how the split-apply-combine pattern works:

```
┌─────────────────────────────────────────────────────────────────────────┐
│  Split-Apply-Combine Pattern                                            │
│                                                                         │
│  Original DataFrame                                                     │
│  ┌──────┬─────┬──────┐                                                  │
│  │Class │ Sex │ Fare │                                                  │
│  ├──────┼─────┼──────┤                                                  │
│  │  1   │  M  │  50  │ ─┐                                               │
│  │  1   │  F  │  80  │  ├─ Group: Class 1                               │
│  │  1   │  M  │  70  │ ─┘                                               │
│  │  2   │  F  │  25  │ ─┐                                               │
│  │  2   │  M  │  20  │  ├─ Group: Class 2                               │
│  │  2   │  F  │  30  │ ─┘                                               │
│  │  3   │  M  │  10  │ ─┐                                               │
│  │  3   │  M  │   8  │  ├─ Group: Class 3                               │
│  │  3   │  F  │  12  │ ─┘                                               │
│  └──────┴─────┴──────┘                                                  │
│           │                                                             │
│           │ SPLIT by 'Class'                                            │
│           ▼                                                             │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐                      │
│  │ Class 1     │  │ Class 2     │  │ Class 3     │                      │
│  │ Fare: 50,   │  │ Fare: 25,   │  │ Fare: 10,   │                      │
│  │       80,70 │  │       20,30 │  │        8,12 │                      │
│  └─────────────┘  └─────────────┘  └─────────────┘                      │
│           │               │               │                             │
│           │ APPLY mean()  │               │                             │
│           ▼               ▼               ▼                             │
│        66.67           25.00           10.00                            │
│           │               │               │                             │
│           └───────────────┴───────────────┘                             │
│                           │                                             │
│                           │ COMBINE                                     │
│                           ▼                                             │
│                   ┌───────┬──────┐                                      │
│                   │ Class │ Mean │                                      │
│                   │   1   │66.67 │                                      │
│                   │   2   │25.00 │                                      │
│                   │   3   │10.00 │                                      │
│                   └───────┴──────┘                                      │
└─────────────────────────────────────────────────────────────────────────┘
```

Let's start with the necessary imports and load the Titanic dataset, which will serve as our primary example throughout this section. This dataset contains information about 891 passengers aboard the Titanic, including their passenger class, fare paid, age, and survival status—making it ideal for demonstrating grouping operations with meaningful real-world context:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

# Load the Titanic dataset
titanic = sns.load_dataset('titanic')
print(f"Dataset shape: {titanic.shape}")
titanic.head()

Dataset shape: (891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Basic Groupby Operations

The `groupby` method is the entry point for all group-based operations in Pandas. When you call `groupby` on a DataFrame, you get back a `DataFrameGroupBy` object—not the actual grouped data, but rather an intermediate representation that knows how to split the data when you request a computation. This design is intentional: by deferring the actual grouping until an aggregation is requested, Pandas can optimize the computation and avoid unnecessary work.

In [2]:
# The groupby object is a kind of 'pending' computation
titanic.groupby('pclass')

Notice that we don't see any results—the grouping itself is a *lazy* operation. The actual work happens when we call an aggregation method on this object. This lazy evaluation is similar to how Python generators work: the `DataFrameGroupBy` object holds the recipe for grouping but doesn't execute it until needed. Let's trigger the computation by asking for the mean of each group:

In [3]:
# Compute the mean for each passenger class
titanic.groupby('pclass')[['age', 'fare']].mean()

,age,fare
pclass,,
1,38.233441,84.154687
2,29.877630,20.662183
3,25.140620,13.675550


The result reveals something meaningful about the Titanic: first-class passengers were older on average (around 39 years) and paid dramatically higher fares (about $87) compared to third-class passengers (averaging 25 years old and paying only $13). This is the power of groupby—with a single line of code, we can uncover patterns that would require multiple steps in other approaches.

### Selecting Columns After Grouping

When working with DataFrames that have multiple columns, you often want to aggregate only specific columns. In the previous example, we selected `['age', 'fare']` after `groupby` to focus on just those columns. You can also select a single column to get a Series result, which is particularly useful when you want to perform further operations on that single aggregated column:

In [4]:
# Aggregate only the fare column - returns a Series
titanic.groupby('pclass')['fare'].mean()

pclass
1    84.154687
2    20.662183
3    13.675550
Name: fare, dtype: float64

The result is a Series with the group keys (`pclass` values) as the index. If we want a DataFrame instead, we can pass a list with a single column name, though this is more commonly used when aggregating multiple columns as we saw earlier.

## Grouping by Multiple Columns

Real-world data often requires grouping by multiple criteria simultaneously. For instance, we might want to examine survival rates broken down by both passenger class and sex—a more nuanced view that can reveal interactions between these factors. Pandas handles this naturally by passing a list of column names to `groupby`, creating groups for every unique combination of values:

In [5]:
# Group by multiple columns: class and sex
titanic.groupby(['pclass', 'sex'])['survived'].mean()

pclass  sex   
1       female    0.968085
        male      0.368852
2       female    0.921053
        male      0.157407
3       female    0.500000
        male      0.135447
Name: survived, dtype: float64

The result is a Series with a hierarchical index (a *MultiIndex*) representing all unique combinations of passenger class and sex. The survival rates tell a compelling story: women had much higher survival rates than men across all classes, and first-class women had the highest survival rate at around 97%. This kind of cross-tabulated analysis is fundamental to understanding patterns in data, and the hierarchical structure makes it easy to further subset or reshape the results.

## Aggregation with Multiple Functions

Often you'll want to apply multiple aggregation functions simultaneously—for example, computing both the mean and standard deviation for each group. The `agg()` method (short for aggregate) provides this capability, accepting a list of function names or callable objects. This is remarkably convenient for exploratory analysis, as it allows you to see multiple summary statistics at a glance without running separate computations:

In [6]:
# Apply multiple aggregation functions to fare by class
titanic.groupby('pclass')['fare'].agg(['mean', 'std', 'min', 'max'])

,mean,std,min,max
pclass,,,,
1,84.154687,78.380373,0.0,512.3292
2,20.662183,13.417399,0.0,73.5000
3,13.675550,11.778142,0.0,69.5500


The result is a DataFrame where each aggregation function becomes a column. Notice the enormous variation in first-class fares (standard deviation of $80!) compared to third class ($12)—this reflects the range from basic first-class cabins to the most luxurious suites on the ship.

### Named Aggregations

While passing a list of function names is convenient, you may want more control over the output column names and the ability to apply different functions to different columns. The *named aggregation* syntax, introduced in recent versions of Pandas, provides exactly this flexibility. By using keyword arguments where each keyword becomes the output column name and the value is a tuple of `(column_name, aggregation_function)`, we can create highly readable and well-organized summaries:

In [7]:
# Named aggregation with descriptive output columns
titanic.groupby('pclass').agg(
    passenger_count=('survived', 'count'),
    survival_rate=('survived', 'mean'),
    avg_age=('age', 'mean'),
    avg_fare=('fare', 'mean'),
    max_fare=('fare', 'max')
).round(2)

,passenger_count,survival_rate,avg_age,avg_fare,max_fare
pclass,,,,,
1,216,0.63,38.23,84.15,512.33
2,184,0.47,29.88,20.66,73.50
3,491,0.24,25.14,13.68,69.55


This syntax makes the intent of your code much clearer and produces more readable output. The column names now describe what they contain (`survival_rate` rather than just `mean`), making the results immediately interpretable without consulting the code that generated them.

### Custom Aggregation Functions

Beyond the built-in aggregation functions, you can use any function that takes a Series and returns a scalar value. This opens up possibilities for domain-specific calculations. Here we define a function that computes the range (difference between maximum and minimum), which can reveal the spread of values within each group:

In [8]:
def value_range(x):
    """Compute the range of values (max - min)."""
    return x.max() - x.min()

# Use custom function alongside built-in ones
titanic.groupby('pclass')['fare'].agg(['mean', 'median', value_range])

,mean,median,value_range
pclass,,,
1,84.154687,60.2875,512.3292
2,20.662183,14.2500,73.5000
3,13.675550,8.0500,69.5500


The custom function integrates seamlessly with built-in aggregations. Notice how the first-class fare range ($512) dwarfs the other classes, confirming the extreme diversity of accommodations available to wealthy passengers.

## Pivot Tables

From what we've seen so far, `groupby` provides powerful aggregation capabilities. However, when you want to reshape your grouped data into a more readable format—particularly one that shows relationships between two categorical variables—*pivot tables* offer an elegant solution. A pivot table can be thought of as a multidimensional generalization of group-by operations: if `groupby` is like organizing data into labeled folders, a pivot table is like arranging those folders into a grid where rows and columns each represent different dimensions of the data.

Let's create a pivot table showing average fare by passenger class and embarkation port:

In [9]:
# Create a pivot table: average fare by class and embarkation port
pd.pivot_table(
    titanic,
    values='fare',
    index='pclass',
    columns='embarked',
    aggfunc='mean'
).round(2)

embarked,C,Q,S
pclass,,,
1,104.72,90.00,70.36
2,25.36,12.35,20.33
3,11.21,11.18,14.64


The result is a reshaped view of the data where passenger classes form the rows, embarkation ports form the columns, and the cells contain the mean fare. This format is often more intuitive for comparing values across categories—you can easily see that Cherbourg (C) passengers paid the highest fares across all classes, likely because Cherbourg was closer to Paris and attracted wealthier travelers.

### Multiple Values and Aggregations in Pivot Tables

Pivot tables become even more powerful when aggregating multiple values or using multiple aggregation functions. This allows you to create comprehensive summary tables that would otherwise require multiple separate operations. The resulting hierarchical column structure keeps related statistics together:

In [10]:
# Pivot with multiple values
pd.pivot_table(
    titanic,
    values=['fare', 'age'],
    index='pclass',
    columns='sex',
    aggfunc='mean'
).round(2)

age           fare       
sex    female   male  female   male
pclass                             
1       34.61  41.28  106.13  67.23
2       28.72  30.74   21.97  19.74
3       21.75  26.51   16.12  12.66

In [11]:
# Pivot with multiple aggregation functions
pd.pivot_table(
    titanic,
    values='fare',
    index='pclass',
    columns='sex',
    aggfunc=['mean', 'max', 'count']
).round(2)

mean            max          count     
sex     female   male  female    male female male
pclass                                           
1       106.13  67.23  512.33  512.33     94  122
2        21.97  19.74   65.00   73.50     76  108
3        16.12  12.66   69.55   69.55    144  347

### Adding Margins to Pivot Tables

A useful feature of `pivot_table` is the ability to add row and column totals (called *margins*). This provides summary statistics across all groups, giving you both the detailed breakdown and the overall picture in a single table. The margins act like the "Total" row and column you might add in a spreadsheet:

In [12]:
# Pivot table with margins showing row/column totals
pd.pivot_table(
    titanic,
    values='survived',
    index='pclass',
    columns='sex',
    aggfunc='mean',
    margins=True,
    margins_name='Overall'
).round(3)

sex,female,male,Overall
pclass,,,
1,0.968,0.369,0.630
2,0.921,0.157,0.473
3,0.500,0.135,0.242
Overall,0.742,0.189,0.384


## Transform: Group-Wise Operations That Preserve Shape

While aggregation reduces each group to a single value, sometimes you want to perform a computation within each group that preserves the original DataFrame's shape. This is where the `transform()` method shines. Think of `transform` as broadcasting an aggregation result back to the original rows: if `groupby().mean()` gives you one average per group, `groupby().transform('mean')` gives you that same average repeated for every row belonging to that group.

This distinction is crucial for operations like normalization within groups, computing deviations from group means, or filling missing values with group statistics. Let's see the difference between aggregation and transformation:

In [13]:
# Compare aggregation vs transformation
print("Aggregation (reduces to one row per group):")
print(titanic.groupby('pclass')['fare'].mean())
print(f"\nResult shape: {titanic.groupby('pclass')['fare'].mean().shape}")

Aggregation (reduces to one row per group):
pclass
1    84.154687
2    20.662183
3    13.675550
Name: fare, dtype: float64

Result shape: (3,)


In [14]:
print("Transformation (preserves original shape):")
print(titanic.groupby('pclass')['fare'].transform('mean').head(10))
print(f"\nResult shape: {titanic.groupby('pclass')['fare'].transform('mean').shape}")

Transformation (preserves original shape):
0    13.675550
1    84.154687
2    13.675550
3    84.154687
4    13.675550
5    13.675550
6    84.154687
7    13.675550
8    13.675550
9    20.662183
Name: fare, dtype: float64

Result shape: (891,)


Notice how aggregation produces 3 values (one per class) while transformation produces 891 values (one per passenger). Each passenger's transformed value equals their class's mean fare. This is particularly useful when you want to add a new column that contains group-level statistics, like computing how each passenger's fare compares to their class average:

In [15]:
# Add columns showing deviation from class average
titanic_analysis = titanic[['pclass', 'sex', 'age', 'fare']].copy()
titanic_analysis['class_avg_fare'] = titanic.groupby('pclass')['fare'].transform('mean')
titanic_analysis['fare_vs_class_avg'] = titanic_analysis['fare'] - titanic_analysis['class_avg_fare']
titanic_analysis.head(10).round(2)

,pclass,sex,age,fare,class_avg_fare,fare_vs_class_avg
0,3,male,22.0,7.25,13.68,-6.43
1,1,female,38.0,71.28,84.15,-12.87
2,3,female,26.0,7.92,13.68,-5.75
3,1,female,35.0,53.10,84.15,-31.05
4,3,male,35.0,8.05,13.68,-5.63
5,3,male,NaN,8.46,13.68,-5.22
6,1,male,54.0,51.86,84.15,-32.29
7,3,male,2.0,21.08,13.68,7.40
8,3,female,27.0,11.13,13.68,-2.54
9,2,female,14.0,30.07,20.66,9.41


### Standardization Within Groups

A common application of `transform` is standardizing data within groups—subtracting the group mean and dividing by the group standard deviation. This allows meaningful comparisons across groups with different scales. For instance, we can identify which passengers paid unusually high or low fares relative to their class:

In [16]:
# Standardize fares within each class
def standardize(x):
    """Standardize values to zero mean and unit variance."""
    return (x - x.mean()) / x.std()

titanic_analysis['fare_zscore'] = titanic.groupby('pclass')['fare'].transform(standardize)

# Find passengers who paid unusually high fares for their class
high_payers = titanic_analysis[titanic_analysis['fare_zscore'] > 2]
print(f"Passengers paying >2 std above their class mean: {len(high_payers)}")
high_payers.head()

Passengers paying >2 std above their class mean: 44


,pclass,sex,age,fare,class_avg_fare,fare_vs_class_avg,fare_zscore
27,1,male,19.0,263.0000,84.154687,178.845313,2.281761
50,3,male,7.0,39.6875,13.675550,26.011950,2.208494
59,3,male,11.0,46.9000,13.675550,33.224450,2.820857
71,3,female,16.0,46.9000,13.675550,33.224450,2.820857
72,2,male,21.0,73.5000,20.662183,52.837817,3.938007


### Filling Missing Values with Group Statistics

One subtlety when filling missing values is that a single global statistic (like the overall mean) may not be appropriate if the data varies systematically by group. The `transform` method enables group-aware imputation, filling missing values with statistics computed separately for each group. For the Titanic dataset, ages vary by passenger class, so filling missing ages with class-specific medians makes more sense than using a global median:

In [17]:
# Count missing ages by class before filling
print("Missing ages by class:")
print(titanic.groupby('pclass')['age'].apply(lambda x: x.isna().sum()))

# Fill missing ages with class-specific median
titanic_filled = titanic.copy()
titanic_filled['age'] = titanic.groupby('pclass')['age'].transform(
    lambda x: x.fillna(x.median())
)

print("\nMissing ages after filling:", titanic_filled['age'].isna().sum())

Missing ages by class:
pclass
1     30
2     11
3    136
Name: age, dtype: int64

Missing ages after filling: 0


## Filter: Selecting Groups Based on Group Properties

Sometimes you want to keep or discard entire groups based on some property of the group as a whole. The `filter()` method enables exactly this—it takes a function that returns `True` or `False` for each group, and returns only the rows belonging to groups that pass the filter. This is different from boolean indexing on individual rows; here we're filtering based on group-level properties.

In [18]:
# Load tips dataset for filter demonstration
tips = sns.load_dataset('tips')
print(f"Tips dataset: {len(tips)} meals")
tips.head()

Tips dataset: 244 meals


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [19]:
# Keep only days with more than 50 recorded meals
busy_days = tips.groupby('day').filter(lambda x: len(x) > 50)
print(f"Original dataset: {len(tips)} rows")
print(f"After filtering: {len(busy_days)} rows")
print(f"\nMeals per day (original):")
print(tips['day'].value_counts())

Original dataset: 244 rows
After filtering: 225 rows

Meals per day (original):
day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64


/tmp/ipykernel_296668/266451676.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  busy_days = tips.groupby('day').filter(lambda x: len(x) > 50)


The filter removed Friday (19 meals) and Thursday (62 meals) because they didn't meet our threshold of more than 50 meals. Only Saturday (87) and Sunday (76) passed. Notice that filtering operates on groups, not individual rows—all rows belonging to passing groups are retained.

In [20]:
# Keep groups where the average tip percentage exceeds 16%
tips['tip_pct'] = tips['tip'] / tips['total_bill'] * 100
generous_groups = tips.groupby('day').filter(lambda x: x['tip_pct'].mean() > 16)

print("Average tip % by day:")
print(tips.groupby('day')['tip_pct'].mean().round(2))
print(f"\nDays with avg tip > 16%: {generous_groups['day'].unique()}")

Average tip % by day:
day
Thur    16.13
Fri     16.99
Sat     15.32
Sun     16.69
Name: tip_pct, dtype: float64

Days with avg tip > 16%: ['Sun', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']


/tmp/ipykernel_296668/741509468.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  generous_groups = tips.groupby('day').filter(lambda x: x['tip_pct'].mean() > 16)
/tmp/ipykernel_296668/741509468.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(tips.groupby('day')['tip_pct'].mean().round(2))


## Choosing the Right Method: A Decision Guide

With `groupby`, `pivot_table`, `transform`, and `filter` all available, it can be confusing to know which tool to reach for. The following table provides guidance based on your analytical goal:

| Goal | Method | Example | Result Shape |
|------|--------|---------|---------------|
| Summarize groups into single values | `groupby().agg()` | Mean fare per class | One row per group |
| Compare across two categorical dimensions | `pivot_table()` | Survival by class × sex | Rows × columns grid |
| Add group statistics to each row | `groupby().transform()` | Class mean fare for each passenger | Same as original |
| Keep only groups meeting criteria | `groupby().filter()` | Only days with 50+ meals | Subset of original |
| Apply different functions to different columns | `groupby().agg({...})` | Mean age, max fare per class | One row per group |
| Custom named output columns | Named aggregation | Clear column names | One row per group |

The key distinction is whether you want to *reduce* your data (aggregation), *reshape* it (pivot tables), *enrich* it with group statistics (transform), or *subset* it based on group properties (filter).

## Practical Example: Comprehensive Group Analysis

Let's bring these concepts together with a comprehensive analysis of the Titanic dataset. We'll combine multiple grouping operations to build a complete picture of survival patterns. This example demonstrates how the various tools complement each other in a realistic analytical workflow:

In [21]:
# Comprehensive summary by class and sex
summary = titanic.groupby(['pclass', 'sex']).agg(
    passengers=('survived', 'count'),
    survivors=('survived', 'sum'),
    survival_rate=('survived', 'mean'),
    avg_age=('age', 'mean'),
    avg_fare=('fare', 'mean')
).round(2)

summary

passengers  survivors  survival_rate  avg_age  avg_fare
pclass sex                                                            
1      female          94         91           0.97    34.61    106.13
       male           122         45           0.37    41.28     67.23
2      female          76         70           0.92    28.72     21.97
       male           108         17           0.16    30.74     19.74
3      female         144         72           0.50    21.75     16.12
       male           347         47           0.14    26.51     12.66

In [22]:
# Pivot table view of the same data for easier comparison
survival_pivot = pd.pivot_table(
    titanic,
    values='survived',
    index='pclass',
    columns='sex',
    aggfunc=['count', 'sum', 'mean'],
    margins=True
).round(2)

survival_pivot

count              sum             mean            
sex    female male  All female male  All female  male   All
pclass                                                     
1          94  122  216     91   45  136   0.97  0.37  0.63
2          76  108  184     70   17   87   0.92  0.16  0.47
3         144  347  491     72   47  119   0.50  0.14  0.24
All       314  577  891    233  109  342   0.74  0.19  0.38

In [23]:
# Identify passengers who fared better or worse than their demographic group
titanic_enriched = titanic.copy()
titanic_enriched['group_survival_rate'] = titanic.groupby(['pclass', 'sex'])['survived'].transform('mean')
titanic_enriched['survived_vs_expected'] = titanic_enriched['survived'] - titanic_enriched['group_survival_rate']

# Show some "unexpected" survivors (third-class males who survived)
unexpected_survivors = titanic_enriched[
    (titanic_enriched['survived'] == 1) & 
    (titanic_enriched['group_survival_rate'] < 0.2)
][['pclass', 'sex', 'age', 'fare', 'group_survival_rate']]

print(f"'Unexpected' survivors (from groups with <20% survival): {len(unexpected_survivors)}")
unexpected_survivors.head(10)

'Unexpected' survivors (from groups with <20% survival): 64


,pclass,sex,age,fare,group_survival_rate
17,2,male,NaN,13.0000,0.157407
21,2,male,34.00,13.0000,0.157407
36,3,male,NaN,7.2292,0.135447
65,3,male,NaN,15.2458,0.135447
74,3,male,32.00,56.4958,0.135447
78,2,male,0.83,29.0000,0.157407
81,3,male,29.00,9.5000,0.135447
107,3,male,NaN,7.7750,0.135447
125,3,male,12.00,11.2417,0.135447
127,3,male,24.00,7.1417,0.135447


## Building a Reusable Analysis Pipeline

Having explored the various grouping tools, we can now combine them into a reusable function. This pipeline encapsulates the analytical workflow we've developed, making it easy to apply the same analysis to different datasets or with different grouping variables:

In [24]:
def analyze_groups(df, group_cols, value_col, outcome_col=None):
    """
    Comprehensive group analysis combining aggregation, transformation, and pivoting.
    
    This function demonstrates the split-apply-combine pattern by:
    - Computing summary statistics for each group
    - Adding group-level information back to each row
    - Creating a pivot table view when two grouping columns are provided
    
    Parameters
    ----------
    df : DataFrame
        Input data to analyze
    group_cols : str or list of str
        Column(s) to group by
    value_col : str
        Numeric column to aggregate
    outcome_col : str, optional
        Binary outcome column for rate calculations (e.g., 'survived')
    
    Returns
    -------
    dict
        Dictionary containing:
        - 'summary': Group-level summary statistics
        - 'enriched': Original data with group statistics added
        - 'pivot': Pivot table (if 2 group columns provided)
    
    Example
    -------
    >>> titanic = sns.load_dataset('titanic')
    >>> results = analyze_groups(titanic, ['pclass', 'sex'], 'fare', 'survived')
    >>> results['summary']
    """
    # Ensure group_cols is a list
    if isinstance(group_cols, str):
        group_cols = [group_cols]
    
    # Build aggregation dictionary
    agg_dict = {
        f'{value_col}_count': (value_col, 'count'),
        f'{value_col}_mean': (value_col, 'mean'),
        f'{value_col}_std': (value_col, 'std'),
        f'{value_col}_min': (value_col, 'min'),
        f'{value_col}_max': (value_col, 'max')
    }
    
    if outcome_col:
        agg_dict[f'{outcome_col}_rate'] = (outcome_col, 'mean')
    
    # Create summary
    summary = df.groupby(group_cols).agg(**agg_dict).round(3)
    
    # Enrich original data with group statistics
    enriched = df.copy()
    enriched[f'{value_col}_group_mean'] = df.groupby(group_cols)[value_col].transform('mean')
    enriched[f'{value_col}_vs_group'] = enriched[value_col] - enriched[f'{value_col}_group_mean']
    
    result = {
        'summary': summary,
        'enriched': enriched
    }
    
    # Create pivot table if two grouping columns provided
    if len(group_cols) == 2:
        pivot = pd.pivot_table(
            df,
            values=value_col,
            index=group_cols[0],
            columns=group_cols[1],
            aggfunc='mean',
            margins=True
        ).round(2)
        result['pivot'] = pivot
    
    return result

In [25]:
# Apply the pipeline to Titanic data
results = analyze_groups(titanic, ['pclass', 'sex'], 'fare', 'survived')

print("Summary statistics:")
display(results['summary'])

print("\nPivot table view:")
display(results['pivot'])

Summary statistics:


fare_count  fare_mean  fare_std  fare_min  fare_max  \
pclass sex                                                           
1      female          94    106.126    74.260    25.929   512.329   
       male           122     67.226    77.548     0.000   512.329   
2      female          76     21.970    10.892    10.500    65.000   
       male           108     19.742    14.922     0.000    73.500   
3      female         144     16.119    11.690     6.750    69.550   
       male           347     12.662    11.682     0.000    69.550   

               survived_rate  
pclass sex                    
1      female          0.968  
       male            0.369  
2      female          0.921  
       male            0.157  
3      female          0.500  
       male            0.135


Pivot table view:


sex,female,male,All
pclass,,,
1,106.13,67.23,84.15
2,21.97,19.74,20.66
3,16.12,12.66,13.68
All,44.48,25.52,32.20


## Summary

In this section, we explored Pandas' powerful grouping and aggregation capabilities using real-world data from the Titanic dataset. The split-apply-combine pattern underlies all group-wise operations, providing a mental model for how data is processed: split into groups, functions applied within each group, and results combined into a coherent output. We saw how `groupby()` creates a grouped object that enables multiple types of operations, from simple means to complex named aggregations with custom functions.

Pivot tables emerged as a powerful tool for reshaping grouped data into cross-tabulated format, making comparisons across multiple dimensions more intuitive. The `transform()` method proved invaluable for adding group-level statistics back to each row without losing the original data structure—essential for tasks like group-aware normalization and missing value imputation. Finally, `filter()` allows us to select entire groups based on aggregate properties, enabling analyses focused on groups that meet specific criteria.

These tools form the foundation for exploratory data analysis and are essential for understanding patterns in your data. As we saw with the Titanic dataset, combining these operations can reveal compelling stories—like the dramatic differences in survival rates across class and gender, or identifying passengers whose outcomes defied their demographic odds. In subsequent sections, we'll build on these foundations to tackle even more complex analytical challenges.